In [2]:
import pandas as pd
import numpy as np
from sklearn import cross_validation as cv
from sklearn.metrics.pairwise import pairwise_distances
from surprise import KNNBasic
algo = KNNBasic()


/home/user/assign2/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
cols = ['user_id', 'item_id', 'rating', 'timestamp']

In [4]:
user_data = pd.read_table('u.data', sep = '\t', names = cols)

In [5]:
user_data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
train_data, test_data = cv.train_test_split(user_data, test_size = 0.25)

In [7]:
train_data.shape

(75000, 4)

In [6]:
n_users = user_data.user_id.unique().shape[0]
n_users

943

In [7]:
n_items =  user_data.item_id.unique().shape[0]
n_items

1682

In [8]:
train_data_matrix = np.zeros((n_users, n_items))

In [9]:
train_data.head()

,user_id,item_id,rating,timestamp
34840,566,755,2,881651561
29529,321,463,3,879440393
37905,76,56,5,875027739
66017,848,490,5,887043514
68390,741,194,4,891457242


In [10]:
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

In [11]:
test_data_matrix = np.zeros((n_users, n_items))


In [12]:
for line in test_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

In [13]:
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]


In [14]:
user_similarity = pairwise_distances(train_data_matrix, metric = 'cosine')

In [15]:
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [16]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [17]:
item_prediction = predict(train_data_matrix, (1-item_similarity), type='item')
user_prediction = predict(train_data_matrix, (1-user_similarity), type='user')

In [18]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [19]:
(rmse(user_prediction, test_data_matrix))

2.682916934781418

In [20]:
print (str(rmse(user_prediction, test_data_matrix)))
print (str(rmse(item_prediction, test_data_matrix)))


2.682916934781418
2.9484362717850567


In [21]:
item_prediction

array([[ 1.21560307,  1.11693761,  1.14315502, ...,  0.15524015,
         1.11420477,  1.5037109 ],
       [ 0.26625418,  0.15536591,  0.21683346, ...,  0.36303179,
         0.14495846,  0.11693814],
       [ 0.10855542,  0.08034748,  0.10507397, ...,  0.59657341,
         0.09008272,  0.03229978],
       ..., 
       [ 0.14758775,  0.09172743,  0.12464073, ...,  0.04157438,
         0.10254288,  0.0868939 ],
       [ 0.39591598,  0.32956602,  0.28190757, ...,  0.2562126 ,
         0.34691859,  0.21783623],
       [ 0.76778773,  0.81187132,  0.75741136, ...,  0.        ,
         0.84284274,  0.8583227 ]])

In [22]:
user_prediction

array([[ 2.69523211,  1.02855273,  0.71474438, ...,  0.27084883,
         0.27682149,  0.27767246],
       [ 1.81499156,  0.25796184,  0.23493818, ..., -0.0641761 ,
        -0.06477416, -0.06563205],
       [ 1.2342326 ,  0.19665872,  0.14191646, ..., -0.08590333,
        -0.09164035, -0.09307737],
       ..., 
       [ 2.28350935,  0.27870273,  0.22722929, ..., -0.16848565,
        -0.165704  , -0.16668048],
       [ 1.96457986,  0.51224185,  0.22792838, ..., -0.07327901,
        -0.07002751, -0.07204701],
       [ 2.52774595,  0.89656299,  0.49761305, ...,  0.0144919 ,
         0.02202263,  0.02156005]])

In [23]:
user_data.columns


Index(['user_id', 'item_id', 'rating', 'timestamp'], dtype='object')

In [28]:
predict = algo.predict(user_data.user_id, user_data.item_id,user_data.rating)

AttributeError: 'KNNBasic' object has no attribute 'trainset'